In [ ]:
import os, sys
import datetime
import time
from colorama import init
init()
from colorama import Fore, Back, Style

from src.learning_lib.parsers.findata_parsers.binance.realtime import BinanceRealtimeParser, concat
from src.learning_lib.parsers.news_parsers.coindesk_realtime import CoinDeskRealTimeParser
from src.learning_lib.models.lstm import LSTMModel
from src.learning_lib.models.nlp import NLPModel, RegressionHead
from src.learning_lib.models.merge_predictions import PredictionMerger
from src.learning_lib.utils.strategy_update import StrategyUpdater
from src.server import Server
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
IMPORT_FINDATA = False
IMPORT_NEWSDATA = False

In [3]:
binance_parser = BinanceRealtimeParser()
coin_parser = CoinDeskRealTimeParser()
lstm_model = LSTMModel()
nlp_model = NLPModel(
    model_path="src/learning_lib/models/NLPmodels/NLP_model.pth",
    tokenizer_path="src/learning_lib/models/NLPmodels/tokenizer",
    valid_tickers_list_path="src/learning_lib/models/NLPassets/tokens_names.txt",
    tickers_order_path="src/learning_lib/models/NLPassets/tokens_order.txt"
)
merger = PredictionMerger()
updater = StrategyUpdater()
server = Server()

In [4]:
cols = ["open", "high", "low", "close", "baseVolume", "quoteVolume",
        "numTrades", "takerBuyBaseVolume", "takerBuyQuoteVolume"]

In [5]:
if __name__ == "__main__":
    with open("src/learning_lib/parsers/findata_parsers/tokens_names.txt") as f:
        tick = [line.strip() for line in f.readlines()]
    binance_parser.init(tick, EXPORT=True)
    coin_parser.init(EXPORT=True)

    if IMPORT_NEWSDATA:
        coin_parser._import("coindesk_news.json")
    else:
        print("importing data from coindesk_news.json" + Fore.YELLOW, "skip", Style.RESET_ALL)

    if IMPORT_FINDATA:
        binance_parser._import(None)
    else:
        print(f"importing data from src/data" + Fore.YELLOW, "skip", Style.RESET_ALL)
    
    # server.start()

    num = 10

    try:
        # findata = concat(binance_parser.request(None), tick)
        # findata.to_csv("findata.csv")
        findata = pd.read_csv("findata.csv", index_col="ind")
        print("findata =", findata.shape)
    except Exception as e:
        print(f"parsing last {num} candles..", Fore.RED + "error")
        print(str(e) + Style.RESET_ALL)
    else:
        print(f"parsing last {num} candles..", Fore.GREEN + "done" + Style.RESET_ALL)

    coin_parser.start()
    binance_parser.start()

    time.sleep(1)
    сpred = np.array([.0 for i in range(120)], dtype=np.float32)
    merged = np.zeros((1, 120), dtype=np.float32)
    df = pd.DataFrame([[.0 for i in range(120)]], columns=[t + "_close" for t in tick])
    # print(df)
    d2 = pd.DataFrame([[1.0 for i in range(1080)]], columns=[t + "_" + col for col in cols for t in tick], dtype=np.float32)
    i = 0
    while True:
        x = False
        if coin_parser.done:
            x = True
            d = coin_parser.fetch()
            data = list(map(lambda x: x[2], list(d.values())))[0]
            cpred = nlp_model.predict(data)
            print("cpred =", cpred.shape)
        if binance_parser.done:
            x = True
            data = binance_parser.fetch()
            d = concat(data, tick)
            d.to_csv("data.csv")
            # print(d)
            # print(d2)
            #print(d.info())
            #print([t + "_close" for t in tick])
            #print("#" * 100)
            print("d =", d.shape)
            print("d2 =", d2.shape)
            #print("#" * 100)
            print("df =", df.shape)
            df = pd.Series([(d[t + "_close"] / d2[t + "_close"])[0] for t in tick])
            # print([d[t + "_close"][0] / d2[t + "_close"][0] for t in tick])
            # print(df)
            i += 1
            # print(df.columns)
            updater.update(merged, df.to_numpy())
            d2 = concat(data, tick)
            findata = pd.concat([findata, d])
            findata = findata.iloc[1:]
            pred = lstm_model.predict(findata)
            #print(pred)
            merged = np.append(merged, merger.merge(pred, cpred), axis=0)
            print("merged =", merged.shape)
        if not x:
            time.sleep(1)

importing data from coindesk_news.json skip 
importing data from src/data skip 
findata = (10, 1080)
parsing last 10 candles.. done
started coindesk realtime parserstarted binance realtime parser  

parsing candle: 2025-01-28 09:07:10 - 2025-01-28 09:12:10 
 14%|█▍        | 17/120 [00:29<02:50,  1.65s/it]

KeyboardInterrupt: 

100%|██████████| 120/120 [03:20<00:00,  1.67s/it]
parsing candle: 2025-01-28 09:12:10 - 2025-01-28 09:17:10 
100%|██████████| 120/120 [03:17<00:00,  1.64s/it]
parsing candle: 2025-01-28 09:17:10 - 2025-01-28 09:22:10 
100%|██████████| 120/120 [03:18<00:00,  1.65s/it]
parsing candle: 2025-01-28 09:22:10 - 2025-01-28 09:27:10 
100%|██████████| 120/120 [03:15<00:00,  1.63s/it]
parsing candle: 2025-01-28 09:27:10 - 2025-01-28 09:32:10 
100%|██████████| 120/120 [03:17<00:00,  1.65s/it]
parsing candle: 2025-01-28 09:32:10 - 2025-01-28 09:37:10 
100%|██████████| 120/120 [03:17<00:00,  1.65s/it]
parsing candle: 2025-01-28 09:37:10 - 2025-01-28 09:42:10 
100%|██████████| 120/120 [03:21<00:00,  1.68s/it]
parsing candle: 2025-01-28 09:42:10 - 2025-01-28 09:47:10 
100%|██████████| 120/120 [03:23<00:00,  1.69s/it]
parsing candle: 2025-01-28 09:47:10 - 2025-01-28 09:52:10 
100%|██████████| 120/120 [03:19<00:00,  1.66s/it]
parsing candle: 2025-01-28 09:52:10 - 2025-01-28 09:57:10 
100%|██████████| 12

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/tima/ups_trading/venv/lib/python3.10/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
  File "/home/tima/ups_trading/venv/lib/python3.10/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.10/socket.py", line 967, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/tima/ups_trading/venv/lib/python3.10/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
  File "/home/tima/ups_trading/venv/lib/python3.10/site-packages/urllib3/connectionpool.py", line 488, in _make_request
    raise new

100%|██████████| 120/120 [14:13<00:00,  7.11s/it]  
parsing candle: 2025-01-28 10:46:23 - 2025-01-28 10:51:23 
100%|██████████| 120/120 [03:16<00:00,  1.64s/it]
parsing candle: 2025-01-28 10:51:23 - 2025-01-28 10:56:23 
100%|██████████| 120/120 [03:14<00:00,  1.62s/it]
parsing candle: 2025-01-28 10:56:23 - 2025-01-28 11:01:23 
100%|██████████| 120/120 [03:17<00:00,  1.64s/it]
parsing candle: 2025-01-28 11:01:23 - 2025-01-28 11:06:23 
100%|██████████| 120/120 [03:23<00:00,  1.69s/it]
parsing candle: 2025-01-28 11:06:23 - 2025-01-28 11:11:23 
100%|██████████| 120/120 [03:19<00:00,  1.66s/it]
parsing candle: 2025-01-28 11:11:23 - 2025-01-28 11:16:23 
100%|██████████| 120/120 [03:16<00:00,  1.64s/it]


In [6]:
df

,CELRUSDT_close,BALUSDT_close,QTUMUSDT_close,UNFIUSDT_close,FLMUSDT_close,RAYUSDT_close,AXSUSDT_close,BTCDOMUSDT_close,KLAYUSDT_close,BATUSDT_close,...,SNXUSDT_close,XRPUSDT_close,XEMUSDT_close,CVCUSDT_close,LRCUSDT_close,XLMUSDT_close,REEFUSDT_close,KAVAUSDT_close,BANDUSDT_close,RSRUSDT_close
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 37%|███▋      | 44/120 [01:16<02:12,  1.75s/it]

In [8]:
d

,CELRUSDT_open,CELRUSDT_high,CELRUSDT_low,CELRUSDT_close,CELRUSDT_baseVolume,CELRUSDT_quoteVolume,CELRUSDT_numTrades,CELRUSDT_takerBuyBaseVolume,CELRUSDT_takerBuyQuoteVolume,BALUSDT_open,...,BANDUSDT_takerBuyQuoteVolume,RSRUSDT_open,RSRUSDT_high,RSRUSDT_low,RSRUSDT_close,RSRUSDT_baseVolume,RSRUSDT_quoteVolume,RSRUSDT_numTrades,RSRUSDT_takerBuyBaseVolume,RSRUSDT_takerBuyQuoteVolume
0,0.0135,0.01352,0.01348,0.0135,663072.0,8948.98563,95,222131.0,2998.53128,2.34,...,7544.22011,0.011694,0.011726,0.011666,0.011716,8470404.0,99052.549782,1061,5672544.0,66346.394396


In [9]:
d2

,CELRUSDT_open,CELRUSDT_high,CELRUSDT_low,CELRUSDT_close,CELRUSDT_baseVolume,CELRUSDT_quoteVolume,CELRUSDT_numTrades,CELRUSDT_takerBuyBaseVolume,CELRUSDT_takerBuyQuoteVolume,BALUSDT_open,...,BANDUSDT_takerBuyQuoteVolume,RSRUSDT_open,RSRUSDT_high,RSRUSDT_low,RSRUSDT_close,RSRUSDT_baseVolume,RSRUSDT_quoteVolume,RSRUSDT_numTrades,RSRUSDT_takerBuyBaseVolume,RSRUSDT_takerBuyQuoteVolume
0,0.0135,0.01352,0.01348,0.0135,663072.0,8948.98563,95,222131.0,2998.53128,2.34,...,7544.22011,0.011694,0.011726,0.011666,0.011716,8470404.0,99052.549782,1061,5672544.0,66346.394396


parsing candle: 2025-01-27 22:22:51 - 2025-01-27 22:27:51 
 40%|████      | 48/120 [01:25<02:02,  1.70s/it]

In [ ]:
pd.Series([(d[t + "_close"] / d2[t + "_close"])[0] for t in tick])

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
115    1.0
116    1.0
117    1.0
118    1.0
119    1.0
Length: 120, dtype: float64

 64%|██████▍   | 77/120 [02:12<01:14,  1.72s/it]